# Flanders Make Scientific Conference 2026 Tutorial: Understand, transform, and enrich your tables with LLMs

Demo description...

### Visualize table

In [1]:
import pandas as pd

In [ ]:
# Open aircraft csv 
aircraft_file_path = 'data/sfo-aircraft-tail-numbers-and-models.csv'
lines_to_keep = 100
df = pd.read_csv(aircraft_file_path, index_col=False)
df = df.head(lines_to_keep)[["aircraft_model", "airline", "creation_date"]]
df = df.replace(',', '', regex=True)

aircraft_file_path_simple = 'data/aircraft_models.csv'
df.to_csv(aircraft_file_path_simple, index=False)
df

,aircraft_model,airline,creation_date
0,CRJ-700,Skywest Airlines,2009-09-10T16:33:00.000
1,E175-,Skywest Airlines,2018-04-23T08:34:00.000
2,B747-400,KLM Royal Dutch Airlines,2018-04-14T16:49:00.000
3,B737-700,Southwest Airlines,2018-03-17T17:20:00.000
4,A320-200,United Airlines,2011-08-23T19:20:00.000
...,...,...,...
95,B747-400,Qantas Airways,2009-06-11T02:55:00.000
96,B767-200,Amerijet,2009-11-25T16:10:00.000
97,EMB-120,Skywest Airlines,2014-11-01T12:16:00.000
98,CRJ-200,Skywest Airlines,2014-04-01T08:35:00.000


### Table Understanding with TorchicTab

In [30]:
from torchic_tab_heuristic.task_calculation_wd import generate_annotations 

use_es_index = False

(subject_column, 
 primary_annotations, 
 secondary_annotations, 
 cea, 
 cpa, 
 cta) = generate_annotations(aircraft_file_path_simple, use_es_index)

TorchicTab Initiated!
Loading table...
Table: data/aircraft_models.csv 
    aircraft_model                   airline creation_date
0         CRJ-700          Skywest Airlines    2009-09-10
1           E175-          Skywest Airlines    2018-04-23
2        B747-400  KLM Royal Dutch Airlines    2018-04-14
3        B737-700        Southwest Airlines    2018-03-17
4        A320-200           United Airlines    2011-08-23
..            ...                       ...           ...
95       B747-400            Qantas Airways    2009-06-11
96       B767-200                  Amerijet    2009-11-25
97        EMB-120          Skywest Airlines    2014-11-01
98        CRJ-200          Skywest Airlines    2014-04-01
99       A350-900            Cathay Pacific    2017-12-30

[100 rows x 3 columns] 

Calculating structural annotations...
Calculating semantic annotations...



Candidates searching...



Context score estimation...



CPA calculation...



CEA calculation...



CTA calculation...
Exporti

Export annotations for reproducibility

In [31]:
import json

def save_json(filename, data):
    """
    Save data to a JSON file.
    """
    with open(filename, "w") as f:
        json.dump(data, f, indent=4)

    return


def load_json(filename):
    """
    Load data from a JSON file.
    """
    with open(filename, "r") as f:
        data = json.load(f)
    return data

In [32]:
annotations = {}
annotations["subject_column"] = subject_column
annotations["primary_annotations"] = primary_annotations
annotations["secondary_annotations"] = secondary_annotations
annotations["cea"] = cea
annotations["cpa"] = cpa
annotations["cta"] = cta
annotations["cqa"] = []
save_json("annotations/annotations3.json", annotations)

In [33]:
# Get subject column of table
print(f"The subject column of the table is column: \n {df.iloc[:, subject_column]}")

The subject column of the table is column: 
 0      CRJ-700
1        E175-
2     B747-400
3     B737-700
4     A320-200
        ...   
95    B747-400
96    B767-200
97     EMB-120
98     CRJ-200
99    A350-900
Name: aircraft_model, Length: 100, dtype: object


In [34]:
# Get generic semantic annotations of table
print("Generic column annotations:")
for i in range(len(df.columns)):
    print("\n",df.columns[i])
    print(df.iloc[:, i])
    print(f"Generic semantic type: {secondary_annotations[i]}")

Generic column annotations:

 aircraft_model
0      CRJ-700
1        E175-
2     B747-400
3     B737-700
4     A320-200
        ...   
95    B747-400
96    B767-200
97     EMB-120
98     CRJ-200
99    A350-900
Name: aircraft_model, Length: 100, dtype: object
Generic semantic type: NE

 airline
0             Skywest Airlines
1             Skywest Airlines
2     KLM Royal Dutch Airlines
3           Southwest Airlines
4              United Airlines
                ...           
95              Qantas Airways
96                    Amerijet
97            Skywest Airlines
98            Skywest Airlines
99              Cathay Pacific
Name: airline, Length: 100, dtype: object
Generic semantic type: ORG

 creation_date
0     2009-09-10T16:33:00.000
1     2018-04-23T08:34:00.000
2     2018-04-14T16:49:00.000
3     2018-03-17T17:20:00.000
4     2011-08-23T19:20:00.000
               ...           
95    2009-06-11T02:55:00.000
96    2009-11-25T16:10:00.000
97    2014-11-01T12:16:00.000
98    201

In [35]:
# Get column type annotations
print("Column type annotations:")
for type_annotation in cta:
    column = int(type_annotation[0])
    annotation = type_annotation[1]
    print("\n",df.columns[column])
    print(df.iloc[:, column])
    print(f"Generic semantic type: {secondary_annotations[column]}")
    print(f"Wikidata retrieved annotation: {annotation}")

Column type annotations:

 aircraft_model
0      CRJ-700
1        E175-
2     B747-400
3     B737-700
4     A320-200
        ...   
95    B747-400
96    B767-200
97     EMB-120
98     CRJ-200
99    A350-900
Name: aircraft_model, Length: 100, dtype: object
Generic semantic type: NE
Wikidata retrieved annotation: http://www.wikidata.org/entity/Q15056993

 airline
0             Skywest Airlines
1             Skywest Airlines
2     KLM Royal Dutch Airlines
3           Southwest Airlines
4              United Airlines
                ...           
95              Qantas Airways
96                    Amerijet
97            Skywest Airlines
98            Skywest Airlines
99              Cathay Pacific
Name: airline, Length: 100, dtype: object
Generic semantic type: ORG
Wikidata retrieved annotation: http://www.wikidata.org/entity/Q46970


In [36]:
# Get column relationship annotations
print("Column relationships:")
for rel_annotation in cpa:
    if rel_annotation[2][-3:] == "/P1": continue #unkown relationship
    col1 = int(rel_annotation[0])
    col2 = int(rel_annotation[1])
    relationship = rel_annotation[2]
    print(f"\nColumn: {df.columns[col1]} --> {relationship} --> Column: {df.columns[col2]}")

Column relationships:

Column: aircraft_model --> http://www.wikidata.org/prop/direct/P137 --> Column: airline


In [39]:
# Get cell annotations
print("Cell entity annotations:")
for cell_annotation in cea:
    col = int(cell_annotation[1])
    row = int(cell_annotation[0]) - 1
    annotation = cell_annotation[2]
    print(f"\nColumn: {df.columns[col]}")
    print(f"Cell name: '{df.iloc[row, col]}' --> Annotation: {annotation}")

Cell entity annotations:

Column: aircraft_model
Cell name: 'CRJ-700' --> Annotation: http://www.wikidata.org/entity/Q962677

Column: aircraft_model
Cell name: 'E175-' --> Annotation: None

Column: aircraft_model
Cell name: 'B747-400' --> Annotation: http://www.wikidata.org/entity/Q906937

Column: aircraft_model
Cell name: 'B737-700' --> Annotation: http://www.wikidata.org/entity/Q15623371

Column: aircraft_model
Cell name: 'A320-200' --> Annotation: http://www.wikidata.org/entity/Q15629179

Column: aircraft_model
Cell name: 'SHORTS-360' --> Annotation: http://www.wikidata.org/entity/Q2140324

Column: aircraft_model
Cell name: 'B737-800' --> Annotation: http://www.wikidata.org/entity/Q12476717

Column: aircraft_model
Cell name: 'EMB-120' --> Annotation: http://www.wikidata.org/entity/Q1163407

Column: aircraft_model
Cell name: 'B747-400' --> Annotation: http://www.wikidata.org/entity/Q906937

Column: aircraft_model
Cell name: 'CRJ-700' --> Annotation: http://www.wikidata.org/entity/Q96

### Enrich Table Leveraging Semantic Available Semantic Information

Since we have now matched airplanes and airlines with a large external knowledge base such as Wikidata,         
we can query this knowledge base to find out more information about our data and enrich them. 

Wikidata plane attributes that maybe useful to include:    
- Plane first flight (Wikidata property P606)
- Plane maximum operating altitude (Wikidata property P2254)
- PLane height (Wikidata property P2048)
- Plane length (Wikidata property P2043)

In [40]:
plane_column = 0
plane_entities = []
counter = 0
for cell_annotation in cea:
    
    col = int(cell_annotation[1])
    row = int(cell_annotation[0]) - 1
    annotation = cell_annotation[2]
    if col == 0: 
        counter += 1
        plane_entities.append(annotation)

plane_entities

['http://www.wikidata.org/entity/Q962677',
 None,
 'http://www.wikidata.org/entity/Q906937',
 'http://www.wikidata.org/entity/Q15623371',
 'http://www.wikidata.org/entity/Q15629179',
 'http://www.wikidata.org/entity/Q2140324',
 'http://www.wikidata.org/entity/Q12476717',
 'http://www.wikidata.org/entity/Q1163407',
 'http://www.wikidata.org/entity/Q906937',
 'http://www.wikidata.org/entity/Q962677',
 'http://www.wikidata.org/entity/Q891786',
 'http://www.wikidata.org/entity/Q891786',
 'http://www.wikidata.org/entity/Q6484',
 'http://www.wikidata.org/entity/Q1163407',
 None,
 None,
 'http://www.wikidata.org/entity/Q31190031',
 'http://www.wikidata.org/entity/Q1163407',
 'http://www.wikidata.org/entity/Q15630770',
 'http://www.wikidata.org/entity/Q15624006',
 'http://www.wikidata.org/entity/Q15623371',
 'http://www.wikidata.org/entity/Q15111575',
 'http://www.wikidata.org/entity/Q31190031',
 None,
 'http://www.wikidata.org/entity/Q68189256',
 'http://www.wikidata.org/entity/Q891786',
 'ht

In [41]:
new_plane_properties = ["P606", "P2254", "P2048", "P2043", "P2050", "P1092", "P1083"]
new_plane_property_labels = ["first_flight", 
                             "maximum_operating_altitude(m)", 
                             "height(m)", 
                             "length(m)",
                             "wingspan(m)",
                             "total_produced",
                             "maximum_capacity"]
new_attributes = {property: [] for property in new_plane_property_labels}

In [43]:
#Create the corresponding  SPARQL query 
from SPARQLWrapper import SPARQLWrapper, JSON

def get_wikidata_object(entity, property):
    #Wikidata connection setup
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)
    sparql.addCustomHttpHeader("User-Agent", "MyPythonApp/1.0 (ioannis.dasoulas@kuleuven.be)")

    #Query with a subject entity and object as inputs
    query = f"""
        SELECT ?object WHERE {{
            <{entity}> wdt:{property} ?object .
        }} LIMIT 1
        """
    
    # Query execution
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Results retrieval
    response = sparql.query().convert()
    try:
        results = response['results']['bindings']
        if results:
            return results[0]['object']['value']
        else:
            return None
    except Exception as e:
        print(f"Error querying {entity} for {property}: {e}")
        return None

In [44]:
# Extract new information with the constructed query 
print("Extracting new information for aircraft models...")
for i in range(len(plane_entities)): 
    for j in range(len(new_plane_properties)):
        print(f"\nPlane: {df['aircraft_model'].to_list()[i]}")
        print(f"Extracting {new_plane_property_labels[j].replace('_',' ')} data...")

        if plane_entities[i] is None: 
            print(f"No {new_plane_property_labels[j].replace('_',' ')} data found.")
            new_attributes[new_plane_property_labels[j]].append(None)
            continue

        object = get_wikidata_object(plane_entities[i], new_plane_properties[j])
        if object is None:
            print(f"No {new_plane_property_labels[j].replace('_',' ')} data found.")
            new_attributes[new_plane_property_labels[j]].append(None)
        else:
            print(f"{new_plane_property_labels[j].replace('_',' ')}: {object}")
            new_attributes[new_plane_property_labels[j]].append(object)

Extracting new information for aircraft models...

Plane: CRJ-700
Extracting first flight data...
first flight: 1999-05-27T00:00:00Z

Plane: CRJ-700
Extracting maximum operating altitude(m) data...
maximum operating altitude(m): 12500

Plane: CRJ-700
Extracting height(m) data...
height(m): 7.55

Plane: CRJ-700
Extracting length(m) data...
No length(m) data found.

Plane: CRJ-700
Extracting wingspan(m) data...
No wingspan(m) data found.

Plane: CRJ-700
Extracting total produced data...
No total produced data found.

Plane: CRJ-700
Extracting maximum capacity data...
No maximum capacity data found.

Plane: E175-
Extracting first flight data...
No first flight data found.

Plane: E175-
Extracting maximum operating altitude(m) data...
No maximum operating altitude(m) data found.

Plane: E175-
Extracting height(m) data...
No height(m) data found.

Plane: E175-
Extracting length(m) data...
No length(m) data found.

Plane: E175-
Extracting wingspan(m) data...
No wingspan(m) data found.

Plane

In [47]:
for attribute in new_attributes: 
    df[attribute] = new_attributes[attribute]
df.to_csv(aircraft_file_path_simple, index=False)

aircraft_file_path_enriched = 'data/aircraft_models_enriched.csv'
df.to_csv(aircraft_file_path_enriched, index=False)
df

,aircraft_model,airline,creation_date,first_flight,maximum_operating_altitude(m),height(m),length(m),wingspan(m),total_produced,maximum_capacity
0,CRJ-700,Skywest Airlines,2009-09-10T16:33:00.000,1999-05-27T00:00:00Z,12500,7.55,None,None,None,None
1,E175-,Skywest Airlines,2018-04-23T08:34:00.000,None,None,None,None,None,None,None
2,B747-400,KLM Royal Dutch Airlines,2018-04-14T16:49:00.000,1988-04-29T00:00:00Z,None,19.4,70.66,64.44,694,None
3,B737-700,Southwest Airlines,2018-03-17T17:20:00.000,None,None,None,None,None,None,None
4,A320-200,United Airlines,2011-08-23T19:20:00.000,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
95,B747-400,Qantas Airways,2009-06-11T02:55:00.000,1988-04-29T00:00:00Z,None,19.4,70.66,64.44,694,None
96,B767-200,Amerijet,2009-11-25T16:10:00.000,None,None,None,None,None,None,None
97,EMB-120,Skywest Airlines,2014-11-01T12:16:00.000,1983-07-27T00:00:00Z,9085,None,20,19.78,354,30
98,CRJ-200,Skywest Airlines,2014-04-01T08:35:00.000,1991-05-10T00:00:00Z,None,None,None,None,1021,50


### Transform to Knowledge Graph

In [52]:
#Enrich annotations to account for newly added columns
annotations = load_json("annotations/annotations.json")
new_primary_annotations = ["L","L","L","L","L","L","L"]
new_secondary_annotations = ["DATE","FLOAT", "FLOAT", "FLOAT", "FLOAT", "INT", "INT"]

for i in range(len(new_primary_annotations)):
    annotations["primary_annotations"].append(new_primary_annotations[i])
    annotations["secondary_annotations"].append(new_secondary_annotations[i])

wikidata_prop_prefix = "http://www.wikidata.org/prop/direct/"
for col in range(len(new_plane_properties)):
    new_cpa_annotation = ["0", len(cpa) + col + 1, wikidata_prop_prefix + new_plane_properties[col]]
    annotations["cpa"].append(new_cpa_annotation)
    
save_json("annotations/annotations_new.json", annotations)

In [62]:
import subprocess
import sys

def run_autorml():
    """
    Runs AutoRML via subprocess, simulating the CLI command.
    """
    command = [
        sys.executable, "-m", "autorml",
        "-i", "data/aircraft_models_enriched.csv",
        "-m",
        "-la",
        "-af", "annotations",
        "-ia", "annotations_new.json",
        "-mf", "mappings",            
        "-rf", "rdf",
        "-ds"        
    ]
    
    try:
        # Run the command and capture output
        result = subprocess.run(
            command, 
            check=True, 
            capture_output=True, 
            text=True
        )
        print("AutoRML Output:\n", result.stdout)
    except subprocess.CalledProcessError as e:
        print("Error running AutoRML:\n", e.stderr)

run_autorml()

AutoRML Output:
 AutoRML initialized!

YARRRML mappings succesfully created!
RML mappings succesfully created!

YARRRML and RML mappings succesfully generated!

RDF graph succesfully generated!

